# Open-loop simulations:
## Situation without control

In this notebook the simulations for ** :

- Network topology 
- Symuvia connection 
- Data examination 

## 1. Network topology 

![No Control](../Output/no-control.gif) 

Length of main road 

- Before merge *1000m*, merge zone *100m*, after merge *400m*

Length of onramp road

- Before merge *900m*, merge zone *100m*

#### Parameters

In [32]:
DT = 0.1 # Sample time 

KC = 0.16 # CAV max density 
KH = 0.0896 # HDV max density
VF = 25.0 # Speed free flow
W = 6.25 # Congestion speed 
E  = 25.0*0.3 # Speed drop for relaxation 

GCAV = 1/(KC*W) # Time headway CAV 
GHDV = 1/(KH*W) # Time headway HDV 
SCAV = VF/(KC*W)+1/KC #  Desired space headway CAV 
SHDV = VF/(KH*W)+1/KH #  Desired space headway HDV

dveh_twy = {'CAV': GCAV, 'HDV': GHDV}
dveh_dwy = {'CAV': 1/KC, 'HDV': 1/KH}

U_MAX = 1.5 # Max. Acceleration
U_MIN = -1.5 # Min. Acceleration

## 2. Symuvia connection

Libraries should be charged via `ctypes` module in python:


### Connection with Symuvia

In this case connect to the simulator. First define the `libSymuVia.dylib` file 

In [147]:
import os 
from ctypes import cdll, create_string_buffer, c_int, byref, c_bool


from sqlalchemy import create_engine, MetaData
from sqlalchemy import Table, Column, String, Integer, Float 
from sqlalchemy import insert, delete, select, case, and_

from xmltodict import parse
from collections import OrderedDict, Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Bokeh
from bokeh.plotting import figure, show
from bokeh.sampledata.iris import flowers
from bokeh.io import output_notebook
from bokeh.palettes import Viridis
from bokeh.plotting import figure, show, output_file
output_notebook() 

# Plotly 
import plotly as py 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go 
init_notebook_mode(connected=True)

import matplotlib
from matplotlib import cm

import ipywidgets as widgets

Loading BokehJS ...

#### Load traffic library

In [2]:
dir_path = os.getcwd()
lib_path_name = ('..','Symuvia','Contents','Frameworks','libSymuVia.dylib')
full_name = os.path.join(dir_path,*lib_path_name)
symuvialib = cdll.LoadLibrary(full_name)     

#### Load Traffic network

In [3]:
file_path = ('..', 'Network', 'Merge.xml')
file_name = os.path.join(dir_path, *file_path)
m = symuvialib.SymLoadNetworkEx(file_name.encode('UTF8'))

#### Define Output: Database

Al results are stored in the folder `Output/SymOut.sqlite`. Table for storing results:

1. `traj` stores trajectories in open loop.

In [12]:
engine_path = ('..','Output','SymOut.sqlite')
engine_name = os.path.join(os.path.sep,*engine_path)
engine_full_name = os.path.join(dir_path,*engine_path)
engine_call = 'sqlite://'+engine_name
engine = create_engine(engine_call)
metadata = MetaData()

try: 
    ltbstr = 'Loaded table in: '
    connection = engine.connect()    
    traj = Table('traj', metadata, autoload=True, autoload_with=engine)
    stmt = delete(traj)
    results = connection.execute(stmt)
except:
    ltbstr = 'Loaded table in: '
    traj = Table('traj', metadata,
             Column('ti', Float()),
             Column('id', Integer()),
             Column('type', String(3)),
             Column('tron', String(10)),
             Column('voie', Integer()),
             Column('dst', Float()),
             Column('abs', Float()),
             Column('vit', Float()),
             Column('ldr', Integer()),
             Column('spc', Float()),
             Column('vld', Float()))
    metadata.create_all(engine)
    connection = engine.connect()
finally: 
    print(ltbstr, engine)

Loaded table in:  Engine(sqlite:///../Output/SymOut.sqlite)


#### Symuvia parsers 

This functions are intended to extract particular information from `Symuvia` or to parse information from the simulator, for use within this study. 

1. Pointers: Variables to request data at each time step of the simluation 

2. Parsers: Data format converters 

3. V2V information: Information required to deploy the control strategy


In [13]:
# Pointers
sRequest = create_string_buffer(10000)
bEnd = c_bool(False)
bSecond = c_bool(True)

In [14]:
def typedict(veh_dict):
    """ 
        Converts dictionary file from xmltodict 
        into numeric formats to be stored in a database
    """
    data = {'id': int(veh_dict['@id']),
        'type': veh_dict['@type'],
        'tron': veh_dict['@tron'],
        'voie': int(veh_dict['@voie']),
        'dst': float(veh_dict['@dst']),
        'abs': float(veh_dict['@abs']),
        'vit': float(veh_dict['@vit']),
       }
    return data 

#### V2V information

Information regarding V2V communicatioin is computed. In particular which is the connectivity, and states derived from this case (*spacing* , *speed leader*)  in this case only a single leader is identified 

In [15]:
# Identify Leader 

def queueveh(dLeader, veh):
    """
        This function creates a queue of vehicles 
        for a particular road segment
    """
    if veh['tron'] in dLeader.keys():
        if  veh['id'] not in dLeader[veh['tron']]:
            dLeader[veh['tron']].append(veh['id'])
    else:
        dLeader[veh['tron']] = [veh['id']]
    return dLeader 

def getlead(dLeader, veh):
    """
        This function identifies the leader of a specific 
        vehicle i
    """
    idx = dLeader[veh['tron']].index(veh['id'])
    if idx != 0:
        return dLeader[veh['tron']][idx-1]
    else: 
        return dLeader[veh['tron']][idx]
        

In [16]:
# Spacing 

def getspace(lTrajVeh):    
    """
        This function obtains spacing between two vehicles 
    """
    # Equilibrium 
    det_eq_s = lambda x: SCAV if x['type']=='CAV' else SHDV
    
    try: 
        # Case single vehicle
        if lTrajVeh['id'] == lTrajVeh['ldr']:            
            return [{'spc':0.0+det_eq_s(lTrajVeh)}] 
        else:
            # Last vehicle
            # Leader out of Network @ ti
            return [{'spc':None}] 
    except (TypeError, IndexError):        
        # Multiple veh @ ti
        space = []
        for veh in lTrajVeh:
            if veh['id'] == veh['ldr']:
                space.append(0.0+det_eq_s(veh))
            else:             
                veh_pos = veh['abs']
                ldr_id = veh['ldr']
                ldr_pos = [ldr['abs'] for ldr in lTrajVeh if ldr['id']==ldr_id]
                if ldr_pos:
                    space.append(ldr_pos[0]-veh_pos)
                else:
                    # Leader out of Network @ ti
                    space.append(0.0)
        space_dct = [{'spc': val} for val in space]
        return space_dct

# Spacing 
    
def getleaderspeed(lTrajVeh):    
    """
        This function obtains speed from the leader. 
    """    
    try: 
        # Case single vehicle
        if lTrajVeh['id'] == lTrajVeh['ldr']:
            return [{'vld': lTrajVeh['vit']}]
        else:
            # Leader out of Network @ ti
            return [{'vld':None}]                     
    except (TypeError, IndexError):        
        # Multiple veh @ ti
        speedldr = []
        for veh in lTrajVeh:
            if veh['id'] == veh['ldr']:
                speedldr.append(veh['vit'])
            else:             
                ldr_id = veh['ldr']
                ldr_vit = [ldr['vit'] for ldr in lTrajVeh if ldr['id']==ldr_id]
                if ldr_vit:
                    speedldr.append(ldr_vit[0])
                else:
                    speedldr.append(veh['vit'])
        speedldr_dct = [{'vld': val} for val in speedldr]
        return speedldr_dct    
    
def updatelist(lTrajVeh,lDict):
    """
        Considering a list of dictionaries as an input
        the funciton updates the parameter given by lDict
    """
    try:
        lTrajVeh.update(lDict[0])
    except AttributeError:
        for d,s in zip(lTrajVeh,lDict):
            d.update(s)
    return lTrajVeh



#### Launch symulation 

In [197]:
progressSim = widgets.FloatProgress(
    value=5,
    min=0,
    max=71.9,
    step=0.1,
    description='Loading:',
    bar_style='info',
    orientation='horizontal'
)

In [201]:
#%%time

N = 800 # Simulation steps

# Start simulation from beginning
m = symuvialib.SymLoadNetworkEx(file_name.encode('UTF8'))

# Clean table
stmt = delete(traj)
results = connection.execute(stmt)

step = iter(range(N)) 
stmt = insert(traj)

t = []

display(progressSim) 

#for step in steps: 
bSuccess = 2 
while bSuccess>0:
    try:
        next(step)
        bSuccess =  symuvialib.SymRunNextStepEx(sRequest, True, byref(bEnd))
        dParsed = parse(sRequest.value.decode('UTF8'))
        ti = dParsed['INST']['@val']
        if dParsed['INST']['TRAJS'] is None:
            pass #print('')
            #print('No vehicles in the network at time: {}'.format(ti))
        else:            
            lVehOD = dParsed['INST']['TRAJS']['TRAJ']      
            lTrajVeh = []
            try:
                lTrajVeh = typedict(lVehOD)
                lTrajVeh['ti'] = ti
                dLeader = {lTrajVeh['tron']: [lTrajVeh['id']]}
                lTrajVeh['ldr'] = getlead(dLeader, lTrajVeh)                  
            except TypeError:
                # Multiple veh @ ti
                for i,  veh in enumerate(lVehOD):
                    TrajVeh = typedict(veh)
                    TrajVeh['ti'] = ti
                    dLeader = queueveh(dLeader, TrajVeh)
                    TrajVeh['ldr'] = getlead(dLeader, TrajVeh)
                    lTrajVeh.append(TrajVeh)
            lSpc = getspace(lTrajVeh)
            lLdrV = getleaderspeed(lTrajVeh)
            lTrajVeh = updatelist(lTrajVeh,lSpc)
            lTrajVeh = updatelist(lTrajVeh,lLdrV)
            results = connection.execute(stmt,lTrajVeh)
            # print('{} vehicles in the network at time: {}'.format(results.rowcount, ti))
          
        t.append(ti)
        progressSim.value = ti           
    except StopIteration:
        print('Stop by iteration')
        print('Last simluation step at time: {}'.format(ti))
        bSuccess = 0
    except:
        print('Return from Symuvia Empty: {}'.format(sRequest.value.decode('UTF8')))
        print('Last simluation step at time: {}'.format(ti))
        bSuccess = 0

FloatProgress(value=71.9, bar_style='info', description='Loading:', max=71.9)

Stop by iteration
Last simluation step at time: 80.00


#### Data examination 

In [117]:
stmt = select([traj])
results = connection.execute(stmt).fetchall()

column_names = traj.columns.keys()
trajDf = pd.DataFrame(results, columns = column_names)
trajDf.head()

trajDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7167 entries, 0 to 7166
Data columns (total 11 columns):
ti      7167 non-null float64
id      7167 non-null int64
type    7167 non-null object
tron    7167 non-null object
voie    7167 non-null int64
dst     7167 non-null float64
abs     7167 non-null float64
vit     7167 non-null float64
ldr     7167 non-null int64
spc     7167 non-null float64
vld     7167 non-null float64
dtypes: float64(6), int64(3), object(2)
memory usage: 616.0+ KB


In [166]:
vehicle_tron = trajDf['tron'].unique().tolist()
vehicle_type = trajDf['type'].unique().tolist()
colormap = plt.cm.get_cmap('viridis')

def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

viridis = matplotlib_to_plotly(colormap, 255)

In [184]:
layout = go.Layout(
    title = 'Trajectories without Control',
    yaxis = dict(
        title = 'Position X [m]'
    ),
    xaxis = dict(
        title = 'Time [s]'
    ),
    width = 900, 
    height = 900,
)


def trace_position_vehicle(traj_type, v_id, vtype):
    """
        Plot trace single vehicle
    """
    dashtrj = {'CAV': 'solid', 'HDV': 'dot'}
    
    trace = go.Scatter(
        x = traj_type['ti'],
        y = traj_type['abs'],
        mode = 'lines',
        name = f'Vehicle {vtype} -  {v_id}',
        line = dict(
            shape = 'spline',
            width = 1,
            dash = dashtrj[vtype]
        )
    )
    
    return trace
        

def update_position_plot(vtype):
    
    traj_type = trajDf[trajDf.type.isin(vtype)]
    traj_id = traj_type.id.unique()
    
    data = [] 
    
    for v in traj_id:    
        traj_veh = traj_type[traj_type.id == v]
        veh_type = traj_veh.type.unique()[0]
        trace_i = trace_position_vehicle(traj_veh, v, veh_type)
        data.append(trace_i)

    fig = go.Figure(data = data, layout = layout)
    iplot(fig)
    

veh_type_wgt = widgets.SelectMultiple(
    options=vehicle_type,
    value=vehicle_type,
    rows=2,
    description='Vehicle type',
    disabled=False
)

widgets.interactive(update_position_plot, vtype=veh_type_wgt)

interactive(children=(SelectMultiple(description='Vehicle type', index=(0, 1), options=('CAV', 'HDV'), rows=2,…

In [192]:
layout = go.Layout(
    title = 'Spacing without Control',
    yaxis = dict(
        title = 'Position X [m]'
    ),
    xaxis = dict(
        title = 'Time [s]'
    ),
    width = 900, 
    height = 900,
)

def trace_space_vehicle(traj_type, v_id, vtype):
    """
        Plot trace single vehicle
    """
    symboltype = {'CAV': 'diamond-x-open', 'HDV': 'x'}
    
    trace = go.Scatter(
        x = traj_type['ti'],
        y = traj_type['spc'],
        mode = 'markers',
        name = f'Vehicle {vtype} -  {v_id}',
        marker = dict(
            size = 5,
            symbol = symboltype[vtype]
        )
    )
    
    return trace
        
def update_space_plot(vtype):
    
    traj_type = trajDf[trajDf.type.isin(vtype)]
    traj_id = traj_type.id.unique()
    
    data = [] 
    
    for v in traj_id:    
        traj_veh = traj_type[traj_type.id == v]
        veh_type = traj_veh.type.unique()[0]
        trace_i = trace_space_vehicle(traj_veh, v, veh_type)
        data.append(trace_i)

    fig = go.Figure(data = data, layout = layout)
    iplot(fig)
    
widgets.interactive(update_space_plot, vtype=veh_type_wgt)

interactive(children=(SelectMultiple(description='Vehicle type', index=(0, 1), options=('CAV', 'HDV'), rows=2,…